In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import avg, round
from pyspark.ml.feature import RFormula, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName("multilinear_regression_car_horsepower").getOrCreate()

cars = spark.read.load('../data/raw/mtcars.csv', format='csv', header=True,
                       inferSchema=True, sep=';')
print("Number of instances in this dataset: ", cars.count())
cars.show()

Number of instances in this dataset: 32
+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+-----+----------+---+
MilesPerGallon|Cylinders|Displacement|RearAxleRatio|Weight|QuarterMileTime|VShapeOrStraightLine|AutomaticOrManual|Gears|Carburetor| HP|
+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+-----+----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39| 2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308| 3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
 181| 6| 225| 276| 346| 2022| 1| 0| 3| 1|105|
 143| 8| 360| 321| 357| 1584| 0| 0| 3| 4|245|
 244| 4| 1467| 369| 319| 20| 1| 0| 4| 2| 62|
 228| 4| 1408| 392| 315| 229| 1| 0| 4| 2| 95|
 192| 6| 1676| 392| 344| 183| 1| 0| 4| 4|123|
 178| 6| 1676| 392| 344| 189| 1| 0| 4| 4|123|
 164| 8| 2758| 307| 407| 174| 0| 0| 3| 3|180|
 173| 8| 2758| 307| 373| 176| 0| 0| 3| 3|180|
 152| 8| 2758| 307| 378| 18| 0| 0| 3| 3|180|
 104| 8| 472| 293| 525| 1798| 0| 0| 3| 4|205|
 104| 8| 460| 3| 5424| 1782| 0| 0| 3| 4|215|
 147| 8| 440| 323| 5345| 1742| 0| 0| 3| 4|230|
 324| 4| 787| 408| 22| 1947| 1| 1| 4| 1| 66|
 304| 4| 757| 493| 1615| 1852| 1| 1| 4| 2| 52|
 339| 4| 711| 422| 1835| 199| 1| 1| 4| 1| 65|
+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+-----+----------+---+
only showing top 20 rows

In [ ]:
assembler = VectorAssembler(inputCols=cars.columns, outputCol='corr_features')
cars_assembled = assembler.transform(cars).select('corr_features')

corr_matrix = Correlation.corr(cars_assembled, 'corr_features')
corr_matrix = corr_matrix.collect()[0]["pearson({})".format('corr_features')].toArray()
corr_matrix = spark.createDataFrame(corr_matrix.tolist(), cars.columns)
corr_matrix.select([round(c, 3).alias(c) for c in corr_matrix.columns]).show()

+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+------+----------+------+
MilesPerGallon|Cylinders|Displacement|RearAxleRatio|Weight|QuarterMileTime|VShapeOrStraightLine|AutomaticOrManual| Gears|Carburetor| HP|
+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+------+----------+------+
 1.0| -0.539| 0.092| 0.493|-0.028| -0.073| 0.725| 0.089| 0.031| -0.623|-0.549|
 -0.539| 1.0| 0.055| -0.258| 0.172| 0.115| -0.811| -0.523|-0.493| 0.527| 0.832|
 0.092| 0.055| 1.0| 0.101|-0.252| -0.468| 0.022| -0.368|-0.138| -0.03|-0.065|
 0.493| -0.258| 0.101| 1.0|-0.414| -0.451| 0.323| 0.226| 0.382| -0.135|-0.111|
 -0.028| 0.172| -0.252| -0.414| 1.0| 0.318| -0.118| -0.195| -0.34| -0.078| 0.069|
 -0.073| 0.115| -0.468| -0.451| 0.318| 1.0| -0.045| -0.174|-0.503| -0.213|-0.025|
 0.725| -0.811| 0.022| 0.323|-0.118| -0.045| 1.0| 0.168| 0.206| -0.57|-0.723|
 0.089| -0.523| -0.368| 0.226|-0.195| -0.174| 0.168| 1.0| 0.794| 0.058|-0.243|
 0.031| -0.493| -0.138| 0.382| -0.34| -0.503| 0.206| 0.794| 1.0| 0.274|-0.126|
 -0.623| 0.527| -0.03| -0.135|-0.078| -0.213| -0.57| 0.058| 0.274| 1.0| 0.75|
 -0.549| 0.832| -0.065| -0.111| 0.069| -0.025| -0.723| -0.243|-0.126| 0.75| 1.0|
+--------------+---------+------------+-------------+------+---------------+--------------------+-----------------+------+----------+------+

In [ ]:
r_formula = RFormula(formula="HP ~ MilesPerGallon + Cylinders + VShapeOrStraightLine + Carburetor")
cars_rf = r_formula.fit(cars).transform(cars)
cars_rf.select('features', 'label').show(10)

cars_train, cars_test = cars_rf.randomSplit([0.75, 0.25])
print("Number of training instances: ", cars_train.count())
print("Number of testing instances: ", cars_test.count())

+-------------------+-----+
 features|label|
+-------------------+-----+
 [21.0,6.0,0.0,4.0]|110.0|
 [21.0,6.0,0.0,4.0]|110.0|
[228.0,4.0,1.0,1.0]| 93.0|
[214.0,6.0,1.0,1.0]|110.0|
[187.0,8.0,0.0,2.0]|175.0|
[181.0,6.0,1.0,1.0]|105.0|
[143.0,8.0,0.0,4.0]|245.0|
[244.0,4.0,1.0,2.0]| 62.0|
[228.0,4.0,1.0,2.0]| 95.0|
[192.0,6.0,1.0,4.0]|123.0|
+-------------------+-----+
only showing top 10 rows

Number of training instances: 25
Number of testing instances: 7

In [ ]:
regressor = LinearRegression(maxIter=1000)
model = regressor.fit(cars_train)

pred = model.transform(cars_test)
pred.select('label', 'prediction').show()

+-----+------------------+
label| prediction|
+-----+------------------+
245.0|210.38020406710996|
123.0|173.79705907761002|
175.0|170.15128624473522|
110.0|113.02117215058179|
 95.0| 80.63430135428739|
113.0| 81.51085534637865|
 66.0| 61.34449521197534|
+-----+------------------+

In [ ]:
print("Regression Evaluation Metrics: ")

evaluator = RegressionEvaluator(metricName='rmse')
print("RMSE: ", evaluator.evaluate(pred))

evaluator.setMetricName('r2')
print("R2: ", evaluator.evaluate(pred))

evaluator.setMetricName('mae')
print("MAE: ", evaluator.evaluate(pred))

evaluator.setMetricName('var')
print("Explained variance: ", evaluator.evaluate(pred))

Regression Evaluation Metrics: 
RMSE: 26.80917284662242
R2: 0.7634385071450772
MAE: 20.54244128624361
Explained variance: 2845.0950394772517